<div style="max-width:66ch;">

# Lecture notes - EDA on public transport API

This is the lecture note for working with **APIs**. We will dive into doing get requests from trafiklab API to get public transport data.

In this lecture we'll cover

- trafiklab API
- working with .env
- working with json data

<p class = "alert alert-info" role="alert"><b>Note</b> that this lecture note gives a brief introduction to API and how to work with it. I encourage you to read further about APIs.

</div>


## Routeplanner

- [ResRobot Route planner API](https://www.trafiklab.se/api/trafiklab-apis/resrobot-v21/route-planner/)

- note that you need a .env file where you will store your API key for ResRobot
- also make sure that your .gitignore includes .env so that it is not pushed to your github repo

In [51]:
from dotenv import load_dotenv
import os
import requests

load_dotenv()


def get_trips(origin_id=740000001, destination_id=740098001):
    """origing_id and destination_id can be found from Stop lookup API"""
    API_KEY = os.getenv("API_KEY")
    url = f"https://api.resrobot.se/v2.1/trip?format=json&originId={origin_id}&destId={destination_id}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    try:
        response = requests.get(url)
        response.raise_for_status()

        return response.json()
    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")


result = get_trips()
result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [9]:
# check how many trips there are
len(result["Trip"])

5

In [16]:
# checks first trip keys
example_trip = result["Trip"][0]
example_trip.keys()

dict_keys(['Origin', 'Destination', 'ServiceDays', 'LegList', 'calculation', 'TripStatus', 'idx', 'tripId', 'ctxRecon', 'duration', 'rtDuration', 'checksum'])

In [18]:
example_trip["Origin"]

{'name': 'Stockholm Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
 'extId': '740000001',
 'lon': 18.058151,
 'lat': 59.330136,
 'routeIdx': 0,
 'prognosisType': 'PROGNOSED',
 'time': '14:07:00',
 'date': '2025-01-06',
 'minimumChangeDuration': 'PT20M'}

In [20]:
example_trip["Destination"]

{'name': 'Göteborg Centralstation',
 'type': 'ST',
 'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
 'extId': '740000002',
 'lon': 11.973479,
 'lat': 57.708895,
 'routeIdx': 4,
 'prognosisType': 'PROGNOSED',
 'time': '17:41:00',
 'date': '2025-01-06',
 'minimumChangeDuration': 'PT15M'}

In [22]:
example_trip["TripStatus"]

{'hintCode': 0}

In [24]:
example_trip["ServiceDays"]

[{'planningPeriodBegin': '2024-12-11',
  'planningPeriodEnd': '2025-06-15',
  'sDaysR': 'inte varje dag',
  'sDaysI': '6. jan t o m 9. jun 2025; 22. mar t o m 19. apr 2025 mån - fre, sön; utom 18. t o m 21. apr 2025, 1. t o m 4. maj 2025, 10., 17., 18., 24. maj, 29. maj t o m 1. jun 2025, 6., 8. jun',
  'sDaysB': '0000007FFFFFFFFFFFFFFFFFFBF7EFDF0FF87DF9F787A800'}]

In [29]:
example_trip["LegList"].keys()

dict_keys(['Leg'])

In [31]:
len(example_trip["LegList"]["Leg"])

1

In [33]:
example_trip["LegList"]["Leg"][0].keys()

dict_keys(['Origin', 'Destination', 'Notes', 'JourneyDetailRef', 'JourneyStatus', 'Product', 'Stops', 'JourneyDetail', 'id', 'idx', 'name', 'number', 'category', 'type', 'reachable', 'waitingState', 'direction', 'directionFlag', 'duration'])

In [48]:
example_stops = example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
example_stops

[{'name': 'Stockholm Centralstation',
  'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
  'extId': '740000001',
  'routeIdx': 0,
  'lon': 18.058151,
  'lat': 59.330136,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '14:07:00',
  'depDate': '2025-01-06',
  'depDir': 'Göteborg Centralstation',
  'minimumChangeDuration': 'PT20M'},
 {'name': 'Katrineholm Centralstation',
  'id': 'A=1@O=Katrineholm Centralstation@X=16208324@Y=58996591@U=1@L=740000166@',
  'extId': '740000166',
  'routeIdx': 1,
  'lon': 16.208324,
  'lat': 58.996591,
  'depTime': '15:00:00',
  'depDate': '2025-01-06',
  'arrTime': '14:59:00',
  'arrDate': '2025-01-06',
  'minimumChangeDuration': 'PT10M'},
 {'name': 'Skövde Centralstation',
  'id': 'A=1@O=Skövde Centralstation@X=13853195@Y=58390898@U=1@L=740000008@',
  'extId': '740000008',
  'routeIdx': 2,
  'lon': 13.853195,
  'lat': 58.390898,
  'depTime': '16:17:00',
  'depDate': '2025-01-06',
  'arrTime': '16:17:00',
  'arrDate': '2025-01

In [50]:
[{stop.get("name"): stop.get("arrTime")} for stop in example_stops]

[{'Stockholm Centralstation': None},
 {'Katrineholm Centralstation': '14:59:00'},
 {'Skövde Centralstation': '16:17:00'},
 {'Herrljunga station': '16:45:00'},
 {'Göteborg Centralstation': '17:41:00'}]

## Stop lookup API to find id

- [ResRobot Stop lookup API](https://www.trafiklab.se/api/trafiklab-apis/resrobot-v21/stop-lookup/)

We will work our way to creating a function to find the id based on location




In [56]:
location = "göteborg"
url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"
response = requests.get(url)
result = response.json()
result.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [69]:
stop_locations = result["stopLocationOrCoordLocation"]
len(stop_locations)

10

In [70]:
stop_locations[0]

{'StopLocation': {'productAtStop': [{'icon': {'res': 'prod_gen'}, 'cls': '2'},
   {'icon': {'res': 'prod_gen'}, 'cls': '4'},
   {'icon': {'res': 'prod_gen'}, 'cls': '8'},
   {'icon': {'res': 'prod_gen'}, 'cls': '16'},
   {'icon': {'res': 'prod_gen'}, 'cls': '64'},
   {'icon': {'res': 'prod_gen'}, 'cls': '128'},
   {'icon': {'res': 'prod_gen'}, 'cls': '256'}],
  'timezoneOffset': 60,
  'id': 'A=1@O=GÖTEBORG@X=11973479@Y=57708895@U=1@L=740098001@B=1@p=1736138202@',
  'extId': '740098001',
  'name': 'GÖTEBORG',
  'lon': 11.973479,
  'lat': 57.708895,
  'weight': 15424,
  'products': 222,
  'meta': True,
  'minimumChangeDuration': 'PT26M'}}

In [71]:
stop_locations[0]["StopLocation"].keys()

dict_keys(['productAtStop', 'timezoneOffset', 'id', 'extId', 'name', 'lon', 'lat', 'weight', 'products', 'meta', 'minimumChangeDuration'])

In [72]:
stop_locations[0]["StopLocation"]["name"]

'GÖTEBORG'

In [84]:
# :<50  -> left align in a 50-character wide field
print(f"{'Stop':<50} {'extId'}")
for stop_location in stop_locations:
    stop = stop_location["StopLocation"]
    print(f"{stop.get('name'):<50} {stop.get('extId')}")

Stop                                               extId
GÖTEBORG                                           740098001
Göteborg Centralstation                            740000002
Göteborg Sävenäs lokstation                        740016365
GÖTEBORG GAMLESTADEN                               740098526
Göteborg Korsvägen                                 740015578
Göteborg Kungsportsplatsen                         740016358
Göteborg Stenpiren                                 740072430
Göteborg Eketrägatan                               740025624
Göteborg Vårväderstorget                           740025707
Göteborg Axel Dahlströms torg                      740025608


In [85]:
def access_id_from_location(location):
    url = f"https://api.resrobot.se/v2.1/location.name?input={location}&format=json&accessId={API_KEY}"

    try:
        response = requests.get(url)
        result = response.json()

        print(f"{'Name':<50} extId")

        for stop in result.get("stopLocationOrCoordLocation"):
            stop_data = next(iter(stop.values()))

            # returns None if extId doesn't exist
            if stop_data.get("extId"):
                print(f"{stop_data['name']:<50} {stop_data['extId']}")

    except requests.exceptions.RequestException as err:
        print(f"Network or HTTP error: {err}")

access_id_from_location("malm")

Name                                               extId
Malm (Gullspång kn)                                740062061
Malmö Centralstation                               740000003
Malmö Triangeln station                            740001587
Malmö Hyllie station                               740001586
Malmö Svågertorp station                           740001546
Malmö Persborg station                             740001486
MALMÖ                                              740098548
Malmö Rosengård station                            740001621
Malmö Östervärn station                            740001483
Malmö Fosieby station                              740001553


## Time tables
- [ResRobot Timetables](https://www.trafiklab.se/api/trafiklab-apis/resrobot-v21/timetables/)

Here I present a few EDAs, I'll leave it to you to make it into a function and find out relevant information

In [88]:
# korsvägen
stop_id=740015578
url = f"https://api.resrobot.se/v2.1/departureBoard?id={stop_id}&format=json&accessId={API_KEY}"

response = requests.get(url)
results = response.json()
results.keys()

dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId'])

In [91]:
departures = results["Departure"]
len(departures)

156

In [94]:
example_departure = departures[0]
example_departure.keys()

dict_keys(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product', 'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat', 'time', 'date', 'reachable', 'direction', 'directionFlag'])

In [99]:
example_departure.get("name"), example_departure.get("direction"), example_departure.get("stop")

('Länstrafik - Buss X4', 'Höga hallar (Härryda kn)', 'Göteborg Korsvägen')

In [97]:
example_departure.get("ProductAtStop")

{'icon': {'res': 'prod_gen'},
 'operatorInfo': {'name': 'Västtrafik',
  'nameS': '279',
  'nameN': '279',
  'nameL': 'Västtrafik',
  'id': '279'},
 'name': 'Länstrafik - Buss X4',
 'internalName': 'Länstrafik - Buss X4',
 'displayNumber': 'X4',
 'num': 'X4',
 'line': 'X4',
 'lineId': '1279520500001',
 'catOut': 'BLT',
 'catIn': 'BLT',
 'catCode': '7',
 'cls': '128',
 'catOutS': 'BLT',
 'catOutL': 'Länstrafik - Buss',
 'operatorCode': '279',
 'operator': 'Västtrafik',
 'admin': '279___',
 'matchId': 'X4;5205;689'}

<div style="background-color: #FFF; color: #212121; border-radius: 1px; width:22ch; box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px; display: flex; justify-content: center; align-items: center;">
<div style="padding: 1.5em 0; width: 70%;">
    <h2 style="font-size: 1.2rem;">Kokchun Giang</h2>
    <a href="https://www.linkedin.com/in/kokchungiang/" target="_blank" style="display: flex; align-items: center; gap: .4em; color:#0A66C2;">
        <img src="https://content.linkedin.com/content/dam/me/business/en-us/amp/brand-site/v2/bg/LI-Bug.svg.original.svg" width="20"> 
        LinkedIn profile
    </a>
    <a href="https://github.com/kokchun/Portfolio-Kokchun-Giang" target="_blank" style="display: flex; align-items: center; gap: .4em; margin: 1em 0; color:#0A66C2;">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png" width="20"> 
        Github portfolio
    </a>
    <span>AIgineer AB</span>
<div>
</div>
